In [ ]:
from keras.layers import Input, concatenate, Dropout,Dense ,Lambda, Flatten, Activation,Lambda
from keras.layers import MaxPooling2D, Convolution2D, AveragePooling2D
from keras.layers import BatchNormalization
from keras.optimizers import SGD,Adam
from keras.models import Model
from keras.datasets.cifar10 import load_data
from tensorflow.keras import backend as K
from matplotlib import pyplot
import keras
from keras.utils import to_categorical
from PIL import Image
import numpy
import tensorflow as tf 

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
class InceptionV_4():
    def __init__(self,num_classes,dropout):
        self.channel_axis=-1
        self.inputs= Input((299,299,3))
        #self.inputs =Lambda(lambda image: keras.backend.(image, (299,299)))(self.inputs)
        self.net = self.Base(self.inputs)
       
        self.net = AveragePooling2D((8,8),border_mode="valid")(self.net)
        self.net = Dropout(dropout)(self.net)
        self.net = Flatten()(self.net)
        self.predictions=Dense(output_dim=num_classes,activation="softmax")(self.net)
        self.model = Model(self.inputs,self.predictions,name="InceptionV_4")
        opt = Adam(lr=0.001)
        self.model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
        
    
    
    def conv2d_apply(self,x, nb_filter, nb_row, nb_col,
        border_mode='same', subsample=(1, 1), bias=False):
        x = Convolution2D(nb_filter, nb_row, nb_col,
                      subsample=subsample,
                      border_mode=border_mode,
                      bias=bias)(x)
        x = BatchNormalization(axis=self.channel_axis)(x)
        x = Activation('relu')(x)
        return x

    def Inception_A(self,input):
        branch_0 = self.conv2d_apply(input, 96, 1, 1)

        branch_1 = self.conv2d_apply(input, 64, 1, 1)
        branch_1 = self.conv2d_apply(branch_1, 96, 3, 3)

        branch_2 = self.conv2d_apply(input, 64, 1, 1)
        branch_2 = self.conv2d_apply(branch_2, 96, 3, 3)
        branch_2 = self.conv2d_apply(branch_2, 96, 3, 3)

        branch_3 = AveragePooling2D((3,3), strides=(1,1), border_mode='same')(input)
        branch_3 = self.conv2d_apply(branch_3, 96, 1, 1)

        x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=self.channel_axis)
        return x
       
    def Reduction_A(self,input):
        branch_0 = self.conv2d_apply(input, 384, 3, 3, subsample=(2,2), border_mode='valid')

        branch_1 = self.conv2d_apply(input, 192, 1, 1)
        branch_1 = self.conv2d_apply(branch_1, 224, 3, 3)
        branch_1 = self.conv2d_apply(branch_1, 256, 3, 3, subsample=(2,2), border_mode='valid')

        branch_2 = MaxPooling2D((3,3), strides=(2,2), border_mode='valid')(input)

        x = concatenate([branch_0, branch_1, branch_2], axis=self.channel_axis)
        return x
       

    def Inception_B(self,input):
        branch_0 = self.conv2d_apply(input, 384, 1, 1)

        branch_1 = self.conv2d_apply(input, 192, 1, 1)
        branch_1 = self.conv2d_apply(branch_1, 224, 1, 7)
        branch_1 = self.conv2d_apply(branch_1, 256, 7, 1)

        branch_2 = self.conv2d_apply(input, 192, 1, 1)
        branch_2 = self.conv2d_apply(branch_2, 192, 7, 1)
        branch_2 = self.conv2d_apply(branch_2, 224, 1, 7)
        branch_2 = self.conv2d_apply(branch_2, 224, 7, 1)
        branch_2 = self.conv2d_apply(branch_2, 256, 1, 7)

        branch_3 = AveragePooling2D((3,3), strides=(1,1), border_mode='same')(input)
        branch_3 = self.conv2d_apply(branch_3, 128, 1, 1)

        x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=self.channel_axis)
        return x
        
    def Reduction_B(self,input):
        
        branch_0 = self.conv2d_apply(input, 192, 1, 1)
        branch_0 = self.conv2d_apply(branch_0, 192, 3, 3, subsample=(2, 2), border_mode='valid')

        branch_1 = self.conv2d_apply(input, 256, 1, 1)
        branch_1 = self.conv2d_apply(branch_1, 256, 1, 7)
        branch_1 = self.conv2d_apply(branch_1, 320, 7, 1)
        branch_1 = self.conv2d_apply(branch_1, 320, 3, 3, subsample=(2,2), border_mode='valid')

        branch_2 = MaxPooling2D((3, 3), strides=(2, 2), border_mode='valid')(input)

        x = concatenate([branch_0, branch_1, branch_2], axis=self.channel_axis)
        return x
   

    def Inception_C(self,input):
        branch_0 = self.conv2d_apply(input, 256, 1, 1)

        branch_1 = self.conv2d_apply(input, 384, 1, 1)
        branch_10 = self.conv2d_apply(branch_1, 256, 1, 3)
        branch_11 = self.conv2d_apply(branch_1, 256, 3, 1)
        branch_1 = concatenate([branch_10, branch_11], axis=self.channel_axis)


        branch_2 = self.conv2d_apply(input, 384, 1, 1)
        branch_2 = self.conv2d_apply(branch_2, 448, 3, 1)
        branch_2 = self.conv2d_apply(branch_2, 512, 1, 3)
        branch_20 = self.conv2d_apply(branch_2, 256, 1, 3)
        branch_21 = self.conv2d_apply(branch_2, 256, 3, 1)
        branch_2 = concatenate([branch_20, branch_21], axis=self.channel_axis)

        branch_3 = AveragePooling2D((3, 3), strides=(1, 1), border_mode='same')(input)
        branch_3 = self.conv2d_apply(branch_3, 256, 1, 1)

        x = concatenate([branch_0, branch_1, branch_2, branch_3], axis=self.channel_axis)
        return x
    
    def Stem(self,input):
        net = self.conv2d_apply(input, 32, 3, 3, subsample=(2,2), border_mode='valid')
        net = self.conv2d_apply(net, 32, 3, 3, border_mode='valid')
        net = self.conv2d_apply(net, 64, 3, 3)

        branch_0 = MaxPooling2D((3,3), strides=(2,2), border_mode='valid')(net)

        branch_1 = self.conv2d_apply(net, 96, 3, 3, subsample=(2,2), border_mode='valid')

        net = concatenate([branch_0, branch_1], axis=self.channel_axis)

        branch_0 = self.conv2d_apply(net, 64, 1, 1)
        branch_0 = self.conv2d_apply(branch_0, 96, 3, 3, border_mode='valid')

        branch_1 = self.conv2d_apply(net, 64, 1, 1)
        branch_1 = self.conv2d_apply(branch_1, 64, 1, 7)
        branch_1 = self.conv2d_apply(branch_1, 64, 7, 1)
        branch_1 = self.conv2d_apply(branch_1, 96, 3, 3, border_mode='valid')

        net = concatenate([branch_0, branch_1], axis=self.channel_axis)

        branch_0 = self.conv2d_apply(net, 192, 3, 3, subsample=(2,2), border_mode='valid')
        branch_1 = MaxPooling2D((3,3), strides=(2,2), border_mode='valid')(net)

        net = concatenate([branch_0, branch_1], axis=self.channel_axis)
        return net
       
    def Base(self,input):
        net = self.Stem(input)
        for idx in range(4):
            net = self.Inception_A(net)

        # 35 x 35 x 384
     
        net = self.Reduction_A(net)

        # 17 x 17 x 1024
        # 7 x Inception-B blocks
        for idx in range(7):
            net = self.Inception_B(net)

        # 17 x 17 x 1024
        # Reduction-B block
        net = self.Reduction_B(net)

        # 8 x 8 x 1536
        # 3 x Inception-C blocks
        for idx in range(3):
            net = self.Inception_C(net)
        return net
        

In [ ]:
inc_v4= InceptionV_4(10,0.8)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), strides=(2, 2), padding="valid", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), strides=(1, 1), padding="valid", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), strides=(1, 1), padding="same", use_bias=False)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:119: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D((3, 3), strides=(2, 2), padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (3, 3), strides=(2, 2), padding="valid", use_bias=False)`
/usr/local/lib/python3.6/dist-packages

In [ ]:
(x_train, y_train), (x_test, y_test) =load_data()

170500096/170498071 [==============================] - 11s 0us/step


In [ ]:

x_train=x_train[:10000]
y_train=y_train[:10000]
# x_test=x_test[:1000]
# y_test=y_test[:1000]

In [ ]:
print('Train: X=%s, y=%s' % (x_train.shape, y_train.shape))
print('Test: X=%s, y=%s' % (x_test.shape, y_test.shape))
# plot first few images
im_res_train=[]
im_res_test=[]
for i in range(x_train.shape[0]):
    img = Image.fromarray(x_train[i])
    img= img.resize((299,299))

    im_res_train.append(numpy.array(img))
# for i in range(x_test.shape[0]):
#     img = Image.fromarray(x_test[i])
#     img= img.resize((299,299))

#     im_res_test.append(numpy.array(img))
    
    
x_train_resize = numpy.array(im_res_train)
# x_test_resize = numpy.array(im_res_test)
print(x_train_resize.shape)
pyplot.show()

Train: X=(10000, 32, 32, 3), y=(10000, 1)
Test: X=(10000, 32, 32, 3), y=(10000, 1)
(10000, 299, 299, 3)


In [ ]:
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


In [ ]:
train_norm = x_train.astype('float32')
test_norm = x_test.astype('float32')
# normalize to range 0-1
train_norm = train_norm / 255.0
test_norm = test_norm / 255.0


In [ ]:

history = inc_v4.model.fit(x_train_resize, y_train ,epochs=100, batch_size=32, verbose=1, validation_split=0.2
                           )

Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 785s 98ms/step - loss: 2.0366 - accuracy: 0.2070 - val_loss: 3.4544 - val_accuracy: 0.2375
Epoch 2/100
8000/8000 [==============================] - 781s 98ms/step - loss: 1.8959 - accuracy: 0.2745 - val_loss: 1.8433 - val_accuracy: 0.3050
Epoch 3/100
8000/8000 [==============================] - 781s 98ms/step - loss: 1.8162 - accuracy: 0.3165 - val_loss: 1.7417 - val_accuracy: 0.3265
Epoch 4/100
8000/8000 [==============================] - 782s 98ms/step - loss: 1.7725 - accuracy: 0.3341 - val_loss: 1.7278 - val_accuracy: 0.3430
Epoch 5/100
8000/8000 [==============================] - 785s 98ms/step - loss: 1.7290 - accuracy: 0.3445 - val_loss: 8.9588 - val_accuracy: 0.2325
Epoch 6/100
8000/8000 [==============================] - 784s 98ms/step - loss: 1.6640 - accuracy: 0.3729 - val_loss: 1.7296 - val_accuracy: 0.3450
Epoch 7/100
8000/8000 [==============================] - 786s 98

In [ ]:
_, acc = inc_v4.model.evaluate(x_test_resize, y_test, verbose=0)

In [ ]:
acc